# **iMaterialist Challenge (Furniture) at FGVC5**
## **TFNW Kaggle Team**


In [6]:
# Import numpy for the high performance in-memory matrix/array storage and operations.
import numpy as np

# Import h5py for the HD5 filesystem high performance file storage of big data.
import h5py

# Import time to record timing
import time


## Reading in the Dataset Batches

We are ready to read in the dataset and feed it to our model. We have stored the dataset as a series of batch files, so we will want to read one batch file at a time and feed it into the model.

### Feeder

The function feeder() performs the feed task for training our model. We use a generator. From wikipedia:

>*A generator is a special routine that can be used to control the iteration behaviour of a loop. A generator is very similar to a function that returns an array, in that a generator has parameters, can be called, and generates a sequence of values.  However, instead of building an array containing all the values and returning them all at once, a generator yields the values one at a time, which requires less memory and allows the caller to get started processing the first few values immediately. In short, a generator looks like a function but behaves like an iterator.*

The feeder() function takes a list of the HD5 stored batch files. Each time the feeder() function is called, it will read in the next batch file, extract the images and corresponding labels, which are returned as the next batch data (X, Y).

Let's take a closer look. The 'while True' may at first appear that the function has an infinite loop and will never return. That's not correct. The key to understanding this is the yield statement. The yield acts similar to a return statement. From Wikipedia:

>*The yield statement is used to define generators, replacing the return of a function to provide a result to its caller without destroying local variables. Unlike a function, where on each call it starts with new set of variables, a generator will resume the execution where it was left off.*

### Feeding Epochs

The 'While True' loop gives the feeder() function the ability to continously cycle through all the batch files. For example, if we have 100 batch files and 2 epochs, the first 100 calls to the feeder will sequentially feed the 100 batch files (see below). The 101st call will cycle back to the while loop and start over again; hence starting the 2nd epoch.

### Feeding Batches within an Epoch

The 'for batch_file in files:' loop is where the batch files are feed one at a time. This loop on each iteration will select the next batch file in the list files.

### Feeding a Batch

The 'with h5py.File(batch_file, 'r') as hf:' statement opens the current batch file and reads in the images and corresponding labels. The file is an HD5 encoding. The images are stored in the group section 'images' and the labels in the group section 'labels'. The HD5 encoding allows random access into the file using indexes. 

To access the 'images' section, we use the associative array access syntax: hf['images']. Likewise, the same for 'labels' section. Within the sections, we copy the entire list of images, and labels using the list copy syntax [:]. For the image data, we normalize the pixel values by converting them from a 0 .. 255 integer range to 0 .. 1 floating point range.

Finally, we get to the yield keyword. At this point the feeder() will return the data read into the variables X and Y, and importantly remember where the function left off. The next time the feeder() function is called, it will pick up where it left off and continue to it reaches the yield statement.

### Note on Normalization

One could ask why we did not store the images in the batch files already normalized. We could have and improve performance in reading in the batch files. 

But it is a trade off for speed vs. disk space. The 0 .. 255 range can be represented as a 8bit integer value, requiring only one byte per pixel in the stored file. If we pre-convert to 0..1, we have a floating point number. At a minimum, we would need to store this value as a 16bit floating point value, requiring two bytes per pixel, doubling the size of the stored file. The 16bit resolution is low precision, so if we have many layers in the neural network, we could face a vanishing gradient problem. In that case, we would need to store as 32bit floating point value, which would require four bytes per pixel, quadrupling the size of the stored file.


In [7]:
def feeder(files):
    """ A generator for feeding batches of images to a neural network 
    files = batches of images/labels as HD5 files
    """
    # We use an infinite loop here so that the generator can be called for an unlimited number of epochs
    while True:
        # Read each batch file one at a time in sequential order
        for batch_file in files:
            # Read the batch to disk from the HD5 file
            with h5py.File(batch_file, 'r') as hf:
                # Read in the images
                # Normalize the pixel data (convert 0..255 to 0..1)
                X = hf['images'][:] / 255
                # Read in the corresponding labels
                Y = hf['labels'][:]
                yield X, Y

### Memory Profiling

When finding the optimal balance between memory (space) and speed (performance), it's generally a good idea to do some memory profiling. We install/import the pympler module for memory profiling.

In [8]:
# Install pympler, used for memory monitoring
!pip install pympler

## Running the Feeder

We are ready now to run the feeder. I have constructed the code below for feeding the training data in batches to a neural network.

### Accessing the Batch Files

We start by getting a list of all the batch files we have stored in our training subdirectory (contents/train). We then calculate the number of batch files by taking the length of this list.

### Running an Epoch

Next, we set the number of epochs (n_epochs) and loop through our epoch range. In each loop iteration, we will feed the entire training data in batches. We do this by keeping a count of the number of batches we have ran. The entire epoch is complete when the count equals the total number of batches (i.e., the number of batch files).

### Running a Batch

Within the epoch loop, we call the iterator on the feeder() function. Each time we call the feeder(), the function will return the images (X) and corresponding labels (Y) from the next batch file.

### Garbage Collection

After each batch is processed (feed into model), and the X and Y data is no longer referenced, Python does not immediately free the memory. Instead, the Python run-time environment does periodic garbage collection (freeing memory), in the background.

The space for X and Y continue to accumulate on the heap until the Python run-time environment implicitly does the garbage collection (i.e. freeing the memory).

If the processing of the data is running much faster than the garbage collection cycle, we could have memory space problems. We are solving this by explicitly calling the garbage collection process after eacy batch is processed.

In [12]:
import os

# Import the garbage (memory management) module
import gc

# Import pympler.tracker for memory monitoring
from pympler import tracker
# Create object to monitor the heap
tr = tracker.SummaryTracker()

# Directory where the training batches are stored
batches = "contents/train/"

# Get a list of all the training\ batch files
batchlist = []
for batch in os.listdir(batches):
    batchlist.append(os.path.join(batches, batch))
    
# Number of batches
nbatches = len(batchlist)

# Number of epochs
n_epochs = 1
    
# Loop through each epoch, each time feeding the entire training set.
for epoch in range(n_epochs):
    # Iteratively call the feeder() function
    nbatch = 0
    for X, Y in feeder(batchlist):
        # Printing some information so you can see that the next batch file was feed
        print("BATCH #:", nbatch)
        print(len(X), X.shape)
        print(len(Y), Y.shape)
        tr.print_diff()
        
        # HERE is where you feed the training batch data to the neural network
        
        # This line will force garbage collection of unused memory.
        # If the processing of the data is running much faster than the garbage collection cycle,
        # we could have memory space problems. We are solving this by explicitly calling the garbage
        # collection process after eacy batch is processed.
        gc.collect()
        
        # Run a single epoch
        nbatch += 1
        if nbatch == nbatches:
            break
    

BATCH #: 0
198 (198, 300, 300, 3)
198 (198,)
                            types |   # objects |   total size
================================= | =========== | ============
            <class 'numpy.ndarray |           0 |     12.36 MB
                     <class 'cell |           4 |    192     B
                     <class 'code |           1 |    144     B
          function (null_wrapper) |           1 |    136     B
            function (store_info) |           1 |    136     B
                <class 'generator |           1 |     88     B
          <class 'h5py.h5f.FileID |           1 |     88     B
                  <class 'weakref |           1 |     80     B
        <class 'functools.partial |           1 |     80     B
                   <class 'method |           1 |     64     B
  <class 'tornado.ioloop._Timeout |           1 |     64     B
      <class 'h5py._hl.files.File |           1 |     56     B
                    <class 'float |           2 |     48     B
          

BATCH #: 18
188 (188, 300, 300, 3)
188 (188,)
                            types |   # objects |       total size
================================= | =========== | ================
          function (null_wrapper) |           1 |        136     B
        <class 'functools.partial |           1 |         80     B
                     <class 'list |           1 |         72     B
                   <class 'method |           1 |         64     B
  <class 'tornado.ioloop._Timeout |           1 |         64     B
                    <class 'float |           2 |         48     B
                      <class 'int |          -1 |        -28     B
   function (_schedule_in_thread) |          -1 |       -136     B
                      <class 'str |          -3 |       -202     B
                     <class 'code |          -2 |       -288     B
                    <class 'tuple |         -14 |       -872     B
                     <class 'cell |         -21 |      -1008     B
              fu